In [1]:
from __future__ import print_function
import time
import os
import torch
import numpy as np
import torch.nn.parallel
import torch.optim as optim
import torchvision.utils as vutils
from torch.autograd import Variable

import re
import sys
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

**Options**

In [2]:
sys.argv = 'xyx_train.py --name xyx_10 --checkpoints_dir ./checkpoints --output_nc 20 --dataset cityscapesAB ' \
        + '--batchSize 4 --heightSize 224 --widthSize 224 --start_epoch 0 --niter 200 ' \
        + '--drop_lr 5 --resize_or_crop crop --ignore_index 0 --unsup_portion 89 --portion_total 90 ' \
        + '--unsup_sampler sep --port 1235 --gpu_ids 1'
sys.argv = sys.argv.split(' ')

In [3]:
from xyx_nets import *

opt = get_opt()
print(opt)

# gpu id
os.environ['CUDA_VISIBLE_DEVICES'] = opt.gpu_ids # absolute ids
if len(opt.gpu_ids) > 0:
    torch.randn(8).cuda()
    os.environ['CUDA_VISIBLE_DEVICES'] = ''
opt.gpu_ids = range(0,len(opt.gpu_ids)) # new range starting from 0

Namespace(archD='patch', archF='style_transform', archG='style_transform', batchSize=4, beta1=0.5, checkpoints_dir='./checkpoints', dataset='cityscapesAB', display_id=1, display_winsize=256, drop_lr=5, gpu_ids='1', heightSize=224, ignore_index=0, input_nc=3, isTrain=True, lambda_x=1.0, lambda_y=1.0, lr=0.0002, lrFGD={'F': 0.0001, 'G': 0.0001, 'D': 0.0001}, manual_seed=123, nThreads=4, n_layers_D=3, n_layers_F=9, n_layers_G=9, name='xyx_10_cityscapesAB_b4/stageF:2,G:0,D:0/lrFGD1e-4,1e-4,1e-4_lbX1.0e+00', ndf=64, ngf=64, niter=200, niter_decay=100, no_flip=False, no_html=False, noise='sphere', nz=100, output_nc=20, port=1235, portion_total=90, resize_or_crop='crop', save_every=2, stage='F:2,G:0,D:0', start_epoch=0, unsup_portion=89, unsup_sampler='sep', updates={'F': 2, 'G': 0, 'D': 0}, use_dropout=False, which_direction='AtoB', widthSize=224)


In [11]:
name_temp1 = '%s/stage%s/lrFGD%s_lbX%.3f'
name_temp2 = '%s/stage%s/lrFGD%s_lbX%.1e'
exp_folder = opt.name.split('/')[0]
print(exp_folder)

xyx_10_cityscapesAB_b4


In [12]:
# losses
CE, L1 = create_losses(opt)

**Data loader**

In [13]:
# data_loaders
from data.data_loader import CreateDataLoader,InfiniteDataLoader,XYDataLoader
from util.visualizer import Visualizer

opt.phase = 'val'
opt.isTrain = False
val_loader = CreateDataLoader(opt)
opt = val_loader.update_opt(opt)

===> dataset [CityscapesABDataset] was created


In [14]:
# networks
TAU0 = 1.0

net =dict()
net['F'] = FX2Y(opt, temperature=TAU0)
net['G'] = GY2X(opt)
net['D'] = NLayerDiscriminator(opt)

for k in net.keys():
    if len(opt.gpu_ids) > 0:
        net[k].cuda(opt.gpu_ids[0])
    # F has its own init
    if k != 'F':
        net[k].apply(weights_init)

def load_nets(ckpt_path):
    for k in net.keys():
        # load if found saved weights
        weights_fpath = os.path.join(ckpt_path, 'net%s.pth' % (k))
        if os.path.exists(weights_fpath):
            print('Load net[%s] from %s' % (k, weights_fpath))
            net[k].load_state_dict(torch.load(weights_fpath, map_location=lambda storage, loc: storage))


In [10]:
from util.meter import SegmentationMeter
def evaluation(epoch, do_G=False, subset='train'):
    heightSize = val_loader.dataset.heightSize
    widthSize = val_loader.dataset.widthSize
    xx = torch.FloatTensor(1, opt.input_nc, heightSize, widthSize)
    yy_int = torch.LongTensor(1, heightSize, widthSize)
    if len(opt.gpu_ids) > 0:
        xx = xx.cuda(opt.gpu_ids[0])
        yy_int = yy_int.cuda(opt.gpu_ids[0])

    net['F'].eval()
    if do_G:
        net['G'].eval()

    eval_stats = SegmentationMeter(n_class=opt.output_nc, ignore_index=opt.ignore_index)
    E_loss_CE = []

    start_time = time.time()
    val_loader_iter = iter(val_loader)
    for i in range(len(val_loader)):
        populate_xy(xx, yy_int, val_loader_iter, opt)
        v_x = Variable(xx, volatile=True)
        v_y_int = Variable(yy_int, volatile=True)
        y_hat = net['F'](v_x)
        E_loss_CE.append( CE(y_hat, v_y_int) )
        logits = y_hat.data.cpu().numpy()
        pred = logits.argmax(1) # NCHW -> NHW
        gt = yy_int.cpu().numpy()
        eval_stats.update_confmat(gt, pred)

        if do_G:
            x_tilde = net['G'](y_hat) # x -> y_hat -> x_tilde

        # visualization
        if i % 10 == 0:
            images = {'TEST_x':v_x.data.cpu(),
                      'TEST_y':v_y_int.data.cpu().numpy(), 'TEST_y_hat':y_hat.data.cpu().numpy().argmax(1)}
            if do_G:
                images['TEST_x_tilde'] = x_tilde.data.cpu()
            display_imgs(images, epoch, i, subset=subset, do_save=2)

    print('EVAL at epoch %d ==> average CE = %.3f' % (epoch, sum(E_loss_CE).data[0] / len(val_loader)))
    eval_results = eval_stats.get_eval_results()
    msg = 'EVAL at epoch %d [%d images in %.2f sec] ==> %s\n' % \
                (epoch, len(val_loader), time.time()-start_time, eval_results[0])
    msg += 'Per-class IoU:\n'
    msg += ''.join(['%s: %.2f\n' % (cname,ciu)
                    for cname,ciu in zip(val_loader.dataset.label2name, eval_results[1])])
    print(msg)
    with open(visualizer.log_name, "a") as log_file:
        log_file.write('%s' % msg)

    net['F'].train()
    net['G'].train()

    return eval_results[0]['Mean IoU']

#-----------------------------------------------------------------------
from util.util import tensor2lab
def display_imgs(images, epoch, i, subset='train', do_save=0):
    if opt.display_id <= 0 and opt.no_html:
        return

    for k, im in images.items():
        if 'y' in k:
            images[k] = tensor2lab(im, val_loader.dataset.label2color) # 3HW
        elif 'x' in k:
            images[k] = im[0] # 3HW
            d_mean = torch.FloatTensor(val_loader.dataset.mean).view(-1,1,1) # (3) -> (3,1,1)
            d_std = torch.FloatTensor(val_loader.dataset.std).view(-1,1,1)
            images[k] *= d_std
            images[k] += d_mean
            images[k] = images[k].mul(255).clamp(0,255).byte().numpy() # 3HW
    visualizer.display_current_results(images, epoch, i, subset=subset, do_save=do_save)

**StageF: F:2,G:0,D:0**

In [18]:
# stage F
stage_str = 'F:2,G:0,D:0'
lr_F = 1e-4
lr_GD = 1e-4
lrFGD = '%.1e,%.1e,%.1e' % (lr_F, lr_GD, lr_GD)
lb = 1.0

name_temp = name_temp1 if lb >= 1e-2 else name_temp2
opt.name =  name_temp % (exp_folder, stage_str, lrFGD, lb)
ckpt_path = './checkpoints/%s' % opt.name

load_nets(ckpt_path) # load net ckpts

# Visualizer
visualizer = Visualizer(opt) # recreate it since opt.name changes

evaluation(999, do_G=False, subset='val')
visualizer.save_webpage(prefix='val')

Load net[F] from ./checkpoints/xyx_10_cityscapesAB_b4/stageF:2,G:0,D:0/lrFGD1.0e-04,1.0e-04,1.0e-04_lbX1.000/netF.pth
===> Create results directory: ./checkpoints/xyx_10_cityscapesAB_b4/stageF:2,G:0,D:0/lrFGD1.0e-04,1.0e-04,1.0e-04_lbX1.000/results
==> OPTIONS:
xyx_train.py --name xyx_10 --checkpoints_dir ./checkpoints --output_nc 20 --dataset cityscapesAB --batchSize 4 --heightSize 224 --widthSize 224 --start_epoch 0 --niter 200 --drop_lr 5 --resize_or_crop crop --ignore_index 0 --unsup_portion 89 --portion_total 90 --unsup_sampler sep --port 1235 --gpu_ids 1



/home/hushell/anaconda3/lib/python3.6/site-packages/torch/nn/modules/container.py:67: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


EVAL at epoch 999 ==> average CE = 5.102
EVAL at epoch 999 [500 images in 19.51 sec] ==> {'Overall Acc': 0.8027315087968822, 'Mean Acc': 0.29677005481120045, 'FreqW Acc': 0.6879863908231599, 'Mean IoU': 0.23759327801778088}
Per-class IoU:
unlabeled: nan
road: 0.89
sidewalk: 0.41
building: 0.66
wall: 0.03
fence: 0.03
pole: 0.08
traffic light: 0.01
traffic sign: 0.03
vegetation: 0.69
terrain: 0.13
sky: 0.74
person: 0.14
rider: 0.00
car: 0.57
truck: 0.02
bus: 0.04
train: 0.00
motorcycle: 0.00
bicycle: 0.05



/space_sdd/hus/pytorch_dir/semisup_semantic/util/meter.py:27: RuntimeWarning: invalid value encountered in true_divide
  acc_cls = np.diag(hist) / hist.sum(axis=1)
/space_sdd/hus/pytorch_dir/semisup_semantic/util/meter.py:29: RuntimeWarning: invalid value encountered in true_divide
  iu = np.diag(hist) / (hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist))


In [11]:
# stage GD
stage_str = 'F:1,G:2,D:2'
lr_F = 1e-4
lr_GD = 1e-4
lrFGD = '%.1e,%.1e,%.1e' % (lr_F, lr_GD, lr_GD)
lb = 1.0

name_temp = name_temp1 if lb >= 1e-2 else name_temp2
opt.name =  name_temp % (exp_folder, stage_str, lrFGD, lb)
ckpt_path = './checkpoints/%s' % opt.name
print(ckpt_path)

load_nets(ckpt_path) # load net ckpts

# Visualizer
visualizer = Visualizer(opt) # recreate it since opt.name changes

evaluation(999, do_G=True, subset='val')
visualizer.save_webpage(prefix='val')

./checkpoints/xyx_10_cityscapesAB_b4/stageF:1,G:2,D:2/lrFGD1.0e-04,1.0e-04,1.0e-04_lbX1.000
Load net[F] from ./checkpoints/xyx_10_cityscapesAB_b4/stageF:1,G:2,D:2/lrFGD1.0e-04,1.0e-04,1.0e-04_lbX1.000/netF.pth
Load net[G] from ./checkpoints/xyx_10_cityscapesAB_b4/stageF:1,G:2,D:2/lrFGD1.0e-04,1.0e-04,1.0e-04_lbX1.000/netG.pth
Load net[D] from ./checkpoints/xyx_10_cityscapesAB_b4/stageF:1,G:2,D:2/lrFGD1.0e-04,1.0e-04,1.0e-04_lbX1.000/netD.pth
===> Create results directory: ./checkpoints/xyx_10_cityscapesAB_b4/stageF:1,G:2,D:2/lrFGD1.0e-04,1.0e-04,1.0e-04_lbX1.000/results
==> OPTIONS:
xyx_train.py --name xyx_10 --checkpoints_dir ./checkpoints --output_nc 20 --dataset cityscapesAB --batchSize 4 --heightSize 224 --widthSize 224 --start_epoch 0 --niter 200 --drop_lr 5 --resize_or_crop crop --ignore_index 0 --unsup_portion 89 --portion_total 90 --unsup_sampler sep --port 1235 --gpu_ids 1



/home/hushell/anaconda3/lib/python3.6/site-packages/torch/nn/modules/container.py:67: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


EVAL at epoch 999 ==> average CE = 5.102
EVAL at epoch 999 [500 images in 32.93 sec] ==> {'Overall Acc': 0.8027315087968822, 'Mean Acc': 0.29677005481120045, 'FreqW Acc': 0.6879863908231599, 'Mean IoU': 0.23759327801778088}
Per-class IoU:
unlabeled: nan
road: 0.89
sidewalk: 0.41
building: 0.66
wall: 0.03
fence: 0.03
pole: 0.08
traffic light: 0.01
traffic sign: 0.03
vegetation: 0.69
terrain: 0.13
sky: 0.74
person: 0.14
rider: 0.00
car: 0.57
truck: 0.02
bus: 0.04
train: 0.00
motorcycle: 0.00
bicycle: 0.05



/space_sdd/hus/pytorch_dir/semisup_semantic/util/meter.py:27: RuntimeWarning: invalid value encountered in true_divide
  acc_cls = np.diag(hist) / hist.sum(axis=1)
/space_sdd/hus/pytorch_dir/semisup_semantic/util/meter.py:29: RuntimeWarning: invalid value encountered in true_divide
  iu = np.diag(hist) / (hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist))


In [12]:
# stage F2
stage_str = 'F:2,G:1,D:1'
lr_F = 1e-4
lr_GD = 1e-4
lrFGD = '%.1e,%.1e,%.1e' % (lr_F, lr_GD, lr_GD)
lb = 1e-2

name_temp = name_temp1 if lb >= 1e-2 else name_temp2
opt.name =  name_temp % (exp_folder, stage_str, lrFGD, lb)
ckpt_path = './checkpoints/%s' % opt.name
print(ckpt_path)

load_nets(ckpt_path) # load net ckpts

# Visualizer
visualizer = Visualizer(opt) # recreate it since opt.name changes

evaluation(999, do_G=True, subset='val')
visualizer.save_webpage(prefix='val')

./checkpoints/xyx_10_cityscapesAB_b4/stageF:2,G:1,D:1/lrFGD1.0e-04,1.0e-04,1.0e-04_lbX0.010
Load net[F] from ./checkpoints/xyx_10_cityscapesAB_b4/stageF:2,G:1,D:1/lrFGD1.0e-04,1.0e-04,1.0e-04_lbX0.010/netF.pth
Load net[G] from ./checkpoints/xyx_10_cityscapesAB_b4/stageF:2,G:1,D:1/lrFGD1.0e-04,1.0e-04,1.0e-04_lbX0.010/netG.pth
Load net[D] from ./checkpoints/xyx_10_cityscapesAB_b4/stageF:2,G:1,D:1/lrFGD1.0e-04,1.0e-04,1.0e-04_lbX0.010/netD.pth
===> Create results directory: ./checkpoints/xyx_10_cityscapesAB_b4/stageF:2,G:1,D:1/lrFGD1.0e-04,1.0e-04,1.0e-04_lbX0.010/results
==> OPTIONS:
xyx_train.py --name xyx_10 --checkpoints_dir ./checkpoints --output_nc 20 --dataset cityscapesAB --batchSize 4 --heightSize 224 --widthSize 224 --start_epoch 0 --niter 200 --drop_lr 5 --resize_or_crop crop --ignore_index 0 --unsup_portion 89 --portion_total 90 --unsup_sampler sep --port 1235 --gpu_ids 1



/home/hushell/anaconda3/lib/python3.6/site-packages/torch/nn/modules/container.py:67: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


EVAL at epoch 999 ==> average CE = 5.698
EVAL at epoch 999 [500 images in 32.54 sec] ==> {'Overall Acc': 0.8085365327797522, 'Mean Acc': 0.2938942303831505, 'FreqW Acc': 0.6916282011055124, 'Mean IoU': 0.24001680924636953}
Per-class IoU:
unlabeled: nan
road: 0.89
sidewalk: 0.40
building: 0.67
wall: 0.03
fence: 0.03
pole: 0.09
traffic light: 0.01
traffic sign: 0.03
vegetation: 0.70
terrain: 0.13
sky: 0.74
person: 0.15
rider: 0.00
car: 0.58
truck: 0.02
bus: 0.05
train: 0.00
motorcycle: 0.00
bicycle: 0.04



/space_sdd/hus/pytorch_dir/semisup_semantic/util/meter.py:27: RuntimeWarning: invalid value encountered in true_divide
  acc_cls = np.diag(hist) / hist.sum(axis=1)
/space_sdd/hus/pytorch_dir/semisup_semantic/util/meter.py:29: RuntimeWarning: invalid value encountered in true_divide
  iu = np.diag(hist) / (hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist))
